# <font color='green'><b> Data Jobs Market Analysis for top 10 US Tech(IT) cities </font></b>

<font size=3>** The aim of this project is to analyse the current job market for data related jobs in US top 10 Tech cities.**

<br><b><u>Job Titles in Scope of the Project</b></u>:<br> 
All Titles with the word 'Data' (includes data analyst, scientist, engineer etc.), Business Intelligence/BI, Artificial Intelligence/AI, Machine Learning, Tableau, Power BI, Statistician/Statistical. 
<br>
<br><u><b>Cities in Scope of the Project</b></u> - Following cities and <b>25</b> miles around them: <br> 
Austin-TX, San Francisco-CA, Raleigh-NC, 
Denver-CO, Seattle-WA, Atlanta-GA, 
Boston-MA, New York City-NY, Washington-D.C., Columbus-OH
</font>

### <font color='green'>Analysis / Questions Addressed</font><br>

<font size=3>

1. Which US city has more data jobs?

2. Citywise which companies are posting more data jobs?

3. Which companies (combined for all 10 cities) are posting relatively higher number of data jobs?

4. What is the number of data jobs getting posted for various job titles like data analyst, data scientist, data engineer, BI, AI, Machine Learning etc.? Are there more jobs for one job title over the other? Which job title has been posted the most and which has been posted the least?

5. What is the salary range for data job wages? How does it change based on the city?

6. Is there a correlation between data job salary and company rating?

7. Is there a correlation between data job salary and minimum experience requirement?

8. Where does the data wages stand in comparison to city and state median wages?

9. How well are the data wages doing in comparison to the apartment rent based on the city?

10. How well are the data wages doing in comparison to the median house prices based on the city?

11. What is the ratio of number of data jobs posted and per capita crime rate based on the city?
</font>


### <font color='green'>Data Sources Used for this module</font><br>

<font size=3>
<ol>
    <li>Jobs data csv created using Indeed API (in data_collection module)</li>
    <li>Salary, rating, experience csv files created using web scraping (in data collection module)</li>
    <li>Zillow - csv files for House Prices</li>
    <li>BLS - csv for City and State Median Salary (All Occupation) data</li>
    <li>Data.World (using FBI data 2015) - csv for Crime data</li>
    <li>Apartment List (Rentonomics) - csv for Apartment Rents data</li>
</ol>
</font>


In [1]:
# import dependencies here

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError
import warnings
import pymysql

In [2]:
import sqlalchemy
sqlalchemy.__version__

'1.3.12'

In [3]:
from plotly import __version__
import plotly as py
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__)  

4.4.1


In [4]:
import cufflinks as cf
# For Notebook
init_notebook_mode(connected=True)
# For offline use
cf.go_offline()

In [5]:
# reading the csv files created using API, we-scraping and downloads

data_jobs = pd.read_csv('../data/all_data_jobs.csv') # API Call
sal_data = pd.read_csv('../data/sal_data.csv') # Web Scraping
rating_data = pd.read_csv('../data/company_ratings_data.csv') # Web Scraping
state_median_sal = pd.read_csv('../data/bls_salary_state.csv', low_memory=False) # csv - BLS
city_median_sal = pd.read_csv('../data/bls_salary_city.csv', low_memory=False) # csv - BLS
us_crime_data = pd.read_csv('../data/US_crime_data_2015.csv') # csv - Data.World (using FBI)
apt_rent_data = pd.read_csv('../data/apartment_rent_data.csv') # csv - Apartment List (Rentonomics)
exp_years = pd.read_csv('../data/experience_data.csv') # Web Scraping


In [6]:
# median home price combining csv files for home price (from zillow)

import glob
path = ('../data/zillow')
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    house_df = pd.read_csv(filename, index_col=None, header=0)
    li.append(house_df)
house_price_df = pd.concat(li, axis=0, ignore_index=True)
house_price_df.dropna(inplace = True)
house_price_data_clean = house_price_df["RegionName"].str.split(",", n = 1, expand = True)
house_price_df["CITY"]= house_price_data_clean[0]
house_price_df["ST_ABBRV"]= house_price_data_clean[1]
house_price_df.drop(columns =["RegionName"], inplace = True)
house_price_df.rename(columns={'SizeRank':'SIZE RANK', 'Bedrooms': 'BEDROOMS_HOUSE', '2019-11':'MEDIAN HOME VALUE'},
                 index={0:1},
                 inplace=True)
median_home_price_df = pd.DataFrame(house_price_df, columns = ["CITY", "ST_ABBRV", "SIZE RANK", "BEDROOMS_HOUSE", "MEDIAN HOME VALUE"])
median_home_price_df.to_csv('../data/median_home_value.csv')

# <font color='green'>'** Data Munging begins here ** </font>

### Cleaning data_jobs dataframe

In [7]:
# checking for total record count
data_jobs.count()

Unnamed: 0               10284
jobtitle                 10284
company                  10283
city                     10272
state                    10284
country                  10284
language                 10284
formattedLocation        10284
source                   10284
date                     10284
snippet                  10284
url                      10284
onmousedown              10284
jobkey                   10284
sponsored                10284
expired                  10284
indeedApply              10284
formattedLocationFull    10284
formattedRelativeTime    10284
stations                     0
dtype: int64

In [8]:
# checking if cities and 25 miles radius search resulted in some extra states 
data_jobs['state'].value_counts()

NY    1800
VA    1633
CA    1375
MA    1250
WA    1075
GA     641
CO     563
DC     468
TX     457
NC     424
MD     299
OH     299
Name: state, dtype: int64

In [9]:
# checking for missing values in the dataframe
data_jobs.isnull().sum()

Unnamed: 0                   0
jobtitle                     0
company                      1
city                        12
state                        0
country                      0
language                     0
formattedLocation            0
source                       0
date                         0
snippet                      0
url                          0
onmousedown                  0
jobkey                       0
sponsored                    0
expired                      0
indeedApply                  0
formattedLocationFull        0
formattedRelativeTime        0
stations                 10284
dtype: int64

In [10]:
# function to replace sub-urban city names(cities within 25 miles of the metropolitan city) with the metropolitan city name

def get_city_name(state):
    
    if state == 'TX':
        return 'Austin'
    if state == 'CA':
        return 'San Francisco'
    if state == 'NC':
        return 'Raleigh'
    if state == 'CO':
        return 'Denver'
    if state == 'WA':
        return 'Seattle'
    if state == 'GA':
        return 'Atlanta'
    if state == 'MA':
        return 'Boston'
    if state == 'NY':
        return 'New York'
    if state == 'OH':
        return 'Columbus'
    if state == 'DC' or 'VA' or 'MD':
        return 'Washington D.C.'


In [11]:
# creating a cleaned dataframe that will replace missing city names using get_city_name function, and 
# for cities within 25 miles of the metropolitan cities, update city name to show that of the metropolitan city for visualization purpose

data_jobs = data_jobs.assign(updated_city=data_jobs['state'].apply(get_city_name)) 

data_jobs['city'].fillna(data_jobs['updated_city'], inplace=True)

data_jobs = data_jobs[['jobtitle', 'company', 'updated_city', 'state',
                       'country', 'source', 'date', 'url', 'jobkey',
                       'sponsored', 'expired', 'indeedApply']] 
 

In [12]:
data_jobs.head(1)

,jobtitle,company,updated_city,state,country,source,date,url,jobkey,sponsored,expired,indeedApply
0,Data Center Security Engineer,"Amazon Dev Center U.S., Inc.",Austin,TX,US,Amazon.com,"Fri, 10 Jan 2020 07:54:39 GMT",http://www.indeed.com/viewjob?jk=389c5707bd0ad...,389c5707bd0addc4,False,False,False


In [13]:
data_jobs.isnull().sum() 

jobtitle        0
company         1
updated_city    0
state           0
country         0
source          0
date            0
url             0
jobkey          0
sponsored       0
expired         0
indeedApply     0
dtype: int64

In [14]:
# further cleaning - dropping the row with blank company name
data_jobs.dropna(inplace=True)

In [15]:
# verifying the clean data
data_jobs.isnull().sum() 

jobtitle        0
company         0
updated_city    0
state           0
country         0
source          0
date            0
url             0
jobkey          0
sponsored       0
expired         0
indeedApply     0
dtype: int64

In [16]:
data_jobs['updated_city'].value_counts()

Washington D.C.    2400
New York           1800
San Francisco      1374
Boston             1250
Seattle            1075
Atlanta             641
Denver              563
Austin              457
Raleigh             424
Columbus            299
Name: updated_city, dtype: int64

### Merging and Cleaning Salary Data

In [17]:
sal_data.head(3) 

,url,annual_min_salary,annual_max_salary
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,50000.0,60000.0
1,http://www.indeed.com/viewjob?jk=f6402472d770b...,75000.0,90000.0
2,http://www.indeed.com/viewjob?jk=d3f99f0101182...,29568.0,29568.0


In [18]:
# merging salary data with jobs data
jobs_sal_data = pd.merge(sal_data, data_jobs, how='inner', on='url')

# selecting required columns
jobs_sal_data = jobs_sal_data[['url', 'jobtitle', 'company', 'annual_min_salary', 'annual_max_salary', 'updated_city', 'state']]

In [19]:
# checking for null values
jobs_sal_data.isnull().sum()

url                  0
jobtitle             0
company              0
annual_min_salary    0
annual_max_salary    0
updated_city         0
state                0
dtype: int64

In [20]:
jobs_sal_data.head(3)

,url,jobtitle,company,annual_min_salary,annual_max_salary,updated_city,state
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,Operations Analyst - Symitar System and Episys...,Austin Telco Federal Credit Union,50000.0,60000.0,Austin,TX
1,http://www.indeed.com/viewjob?jk=f6402472d770b...,Data Base Administrator IV,DEPARTMENT OF INFORMATION RESOURCES,75000.0,90000.0,Austin,TX
2,http://www.indeed.com/viewjob?jk=d3f99f0101182...,CPA - Data Analysis and Transparency Intern,COMPTROLLER OF PUBLIC ACCOUNTS,29568.0,29568.0,Austin,TX


In [21]:
# calculating annual median salaries for data related jobs based on the city

data_jobs_median_sal = jobs_sal_data.groupby('updated_city')[['annual_min_salary', 'annual_max_salary']].agg(np.median) \
                            .reset_index() \
                            .rename(columns={'annual_min_salary': 'annual_min_median', 'annual_max_salary': 'annual_max_median'})
data_jobs_median_sal

,updated_city,annual_min_median,annual_max_median
0,Atlanta,80000.0,110000.0
1,Austin,59004.0,73200.0
2,Boston,75000.0,120000.0
3,Columbus,65780.0,77508.8
4,Denver,63858.0,72624.0
5,New York,62862.0,66388.0
6,Raleigh,49000.0,60320.0
7,San Francisco,100000.0,135100.0
8,Seattle,44400.0,50000.0
9,Washington D.C.,75516.0,114533.0


### Merging and Cleaning Company Ratings Data

In [22]:
rating_data.head(2)

,url,rating
0,http://www.indeed.com/viewjob?jk=389c5707bd0ad...,3.6
1,http://www.indeed.com/viewjob?jk=0df9dac32fbac...,3.8


In [23]:
# merging company ratings data with jobs data

jobs_rating_data = pd.merge(rating_data, data_jobs, how='inner', on='url')

In [24]:
# selecting required columns and dropping duplicate rows (for company rating)

jobs_rating_data = jobs_rating_data[['jobtitle', 'company', 'rating', 'updated_city', 'state']] \
                    .drop_duplicates(subset=['company'], keep='first')
jobs_rating_data.head(2)

,jobtitle,company,rating,updated_city,state
0,Data Center Security Engineer,"Amazon Dev Center U.S., Inc.",3.6,Austin,TX
1,Senior Engineer - Data Science,The Home Depot,3.8,Austin,TX


In [25]:
# checking for null values
jobs_rating_data.isnull().sum()

jobtitle        0
company         0
rating          0
updated_city    0
state           0
dtype: int64

### Cleaning State Annual Median Salary Data

In [26]:
state_median_sal.head(2)

,AREA,ST,STATE,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_Q,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,6,CA,California,00-0000,All Occupations,total,"17,007,690",0.2,1000.0,1.0,...,20.40,35.15,56.54,"23,590","27,830","42,430","73,110","117,590",NaN,NaN
1,8,CO,Colorado,00-0000,All Occupations,total,"2,620,640",0.3,1000.0,1.0,...,20.34,32.49,50.05,"22,480","28,110","42,310","67,570","104,110",NaN,NaN


In [27]:
# assigning city_name column (common across all salary dataframes) needed for comparison and visualization 
state_median_sal = state_median_sal.assign(CITY_NAME=state_median_sal['ST'].apply(get_city_name)) \

# selecting required columns
state_median_sal = state_median_sal[['AREA', 'ST', 'CITY_NAME', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'A_MEDIAN']]

In [28]:
state_median_sal

,AREA,ST,CITY_NAME,OCC_TITLE,OCC_GROUP,TOT_EMP,A_MEDIAN
0,6,CA,San Francisco,All Occupations,total,"17,007,690","42,430"
1,8,CO,Denver,All Occupations,total,"2,620,640","42,310"
2,11,DC,Washington D.C.,All Occupations,total,"712,370","71,690"
3,13,GA,Atlanta,All Occupations,total,"4,394,740","35,950"
4,25,MA,Boston,All Occupations,total,"3,571,360","48,680"
5,36,NY,New York,All Occupations,total,"9,385,620","44,990"
6,37,NC,Raleigh,All Occupations,total,"4,383,210","35,750"
7,39,OH,Columbus,All Occupations,total,"5,416,810","37,360"
8,48,TX,Austin,All Occupations,total,"12,113,810","37,100"
9,53,WA,Seattle,All Occupations,total,"3,259,150","46,100"


In [29]:
# checking for null values
state_median_sal.isnull().sum()

AREA         0
ST           0
CITY_NAME    0
OCC_TITLE    0
OCC_GROUP    0
TOT_EMP      0
A_MEDIAN     0
dtype: int64

### Cleaning City Annual Median Salary Data

In [30]:
# assigning city_name column (common across all salary dataframes) needed for comparison and visualization
city_median_sal = city_median_sal.assign(CITY_NAME=city_median_sal['PRIM_STATE'].apply(get_city_name)) 

# selecting required columns
city_median_sal = city_median_sal[['PRIM_STATE', 'CITY_NAME', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'A_MEDIAN']] 

city_median_sal

,PRIM_STATE,CITY_NAME,OCC_TITLE,OCC_GROUP,TOT_EMP,A_MEDIAN
0,CA,San Francisco,All Occupations,total,"2,419,090","54,090"
1,CO,Denver,All Occupations,total,"1,478,110","45,420"
2,DC,Washington D.C.,All Occupations,total,"3,135,160","54,680"
3,GA,Atlanta,All Occupations,total,"2,682,910","39,080"
4,MA,Boston,All Occupations,total,"2,758,780","51,510"
5,NC,Raleigh,All Occupations,total,"629,570","39,460"
6,NY,New York,All Occupations,total,"9,498,420","47,080"
7,OH,Columbus,All Occupations,total,"1,051,570","39,300"
8,TX,Austin,All Occupations,total,"1,028,500","40,070"
9,WA,Seattle,All Occupations,total,"1,984,320","50,860"


In [31]:
# checking for null values
city_median_sal.isnull().sum()

PRIM_STATE    0
CITY_NAME     0
OCC_TITLE     0
OCC_GROUP     0
TOT_EMP       0
A_MEDIAN      0
dtype: int64

### Cleaning the US Crime Data File

In [32]:
us_crime_data.head(2)

,ORI,year,department_name,total_pop,homs_sum,rape_sum,rob_sum,agg_ass_sum,violent_crime,months_reported,violent_per_100k,homs_per_100k,rape_per_100k,rob_per_100k,agg_ass_per_100k,source,url
0,GAAPD00,2015,Atlanta,464710,94,170,1995,2944,5203,NaN,1119.622991,20.227669,36.581954,429.299993,633.513374,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
1,TX22701,2015,Austin,938728,23,487,929,2058,3497,NaN,372.525375,2.450124,51.878712,98.963704,219.232834,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...


In [33]:
# selecting required columns

us_crime_data = us_crime_data[['department_name', 'total_pop', 'homs_sum', 'rape_sum', 'rob_sum', 'agg_ass_sum', 'violent_crime', 
                               'violent_per_100k', 'homs_per_100k', 'rape_per_100k', 'rob_per_100k', 'agg_ass_per_100k', 'url']] \
                .rename(columns={'department_name': 'city'})    

In [34]:
us_crime_data.head(2)

,city,total_pop,homs_sum,rape_sum,rob_sum,agg_ass_sum,violent_crime,violent_per_100k,homs_per_100k,rape_per_100k,rob_per_100k,agg_ass_per_100k,url
0,Atlanta,464710,94,170,1995,2944,5203,1119.622991,20.227669,36.581954,429.299993,633.513374,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
1,Austin,938728,23,487,929,2058,3497,372.525375,2.450124,51.878712,98.963704,219.232834,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...


In [35]:
# checking for null values
us_crime_data.isnull().sum()

city                0
total_pop           0
homs_sum            0
rape_sum            0
rob_sum             0
agg_ass_sum         0
violent_crime       0
violent_per_100k    0
homs_per_100k       0
rape_per_100k       0
rob_per_100k        0
agg_ass_per_100k    0
url                 1
dtype: int64

### Cleaning Apartment Rent Data

In [36]:
apt_rent_data.head(2)

,Location,Location_Type,State,Bedroom_Size,Price_2014_01,Price_2014_02,Price_2014_03,Price_2014_04,Price_2014_05,Price_2014_06,...,Price_2019_03,Price_2019_04,Price_2019_05,Price_2019_06,Price_2019_07,Price_2019_08,Price_2019_09,Price_2019_10,Price_2019_11,Price_2019_12
0,"New York, NY",City,NY,Studio,1780,1778,1781,1786,1794,1802,...,1889,1904,1920,1936,1933,1928,1922,1937,1940,1943
1,"New York, NY",City,NY,1br,1976,1975,1978,1983,1992,2001,...,2098,2114,2132,2150,2146,2142,2134,2151,2155,2158


In [37]:
# assigning city column (common across all salary dataframes) needed for comparison and visualization
apt_rent_data = apt_rent_data.assign(City=apt_rent_data['State'].apply(get_city_name)) 

# keeping only required columns
apt_rent_data = apt_rent_data[['City', 'State', 'Bedroom_Size', 'Price_2019_12']] 
apt_rent_data = apt_rent_data.pivot(index='City', columns='Bedroom_Size', values='Price_2019_12')

In [38]:
apt_rent_data.head(2)

Bedroom_Size,1br,2br,3br,4br,Studio
City,,,,,
Atlanta,1039,1200,1575,1936,991
Austin,1192,1471,1993,2399,983


### Cleaning Median House Price Data

In [39]:
median_home_price_df.head(2)

,CITY,ST_ABBRV,SIZE RANK,BEDROOMS_HOUSE,MEDIAN HOME VALUE
1,New York,NY,1,1,391567.6667
1,Washington,DC,7,1,312517.6667


In [40]:
# re-arranging the data to show house prices based on size(number of bedrooms)

median_home_price_data = median_home_price_df.pivot(index='CITY', columns='BEDROOMS_HOUSE', values='MEDIAN HOME VALUE')

In [41]:
median_home_price_data

BEDROOMS_HOUSE,1,2,3,4,5
CITY,,,,,
Atlanta,201050.3333,178565.0000,1.979517e+05,2.892450e+05,4.183047e+05
Austin,258838.3333,310509.6667,2.977300e+05,3.856777e+05,5.653223e+05
Boston,350358.0000,382498.0000,4.577677e+05,6.168550e+05,8.774370e+05
Columbus,107595.3333,137180.0000,1.937777e+05,2.991533e+05,4.238153e+05
Denver,250056.6667,342350.0000,4.175047e+05,5.081567e+05,6.139150e+05
New York,391567.6667,340460.0000,4.192160e+05,5.310483e+05,7.738423e+05
Raleigh,173997.6667,173107.0000,2.331973e+05,3.603493e+05,4.889703e+05
San Francisco,NaN,NaN,1.037435e+06,1.283832e+06,1.637889e+06
Seattle,366461.3333,414984.0000,4.746977e+05,6.290253e+05,7.658623e+05


### Merging and Cleaning Experience Data

In [42]:
exp_years.head(2)

,index,url,exp
0,0,http://www.indeed.com/viewjob?jk=389c5707bd0ad...,3
1,1,http://www.indeed.com/viewjob?jk=0df9dac32fbac...,3


In [43]:
# merging experience data with jobs data

jobs_experience_data = pd.merge(exp_years, data_jobs, how='inner', on='url')

In [44]:
# selecting required columns and dropping duplicate rows (for experience)

jobs_experience_data = jobs_experience_data[['jobtitle', 'company', 'exp', 'updated_city', 'state']] \
                    .drop_duplicates(subset=['company'], keep='first')
jobs_experience_data.head(2)

,jobtitle,company,exp,updated_city,state
0,Data Center Security Engineer,"Amazon Dev Center U.S., Inc.",3,Austin,TX
1,Senior Engineer - Data Science,The Home Depot,3,Austin,TX


In [45]:
# checking for null values
jobs_experience_data.isnull().sum()

jobtitle        0
company         0
exp             0
updated_city    0
state           0
dtype: int64

# <br><font color='green'> *** Data ANALYSIS Begins Here *** </font>

## <center><font color='blue'> It's ALL About NUMBERS !!!! The more the Merrier !!</font></center>  

#### Finding Total number of companies combined for all 10 cities that posted data related jobs on the job portal in last 120 days.

In [46]:
# Number of unique companies that posted data related jobs in last 120 days on the job portal  
unique_company_count = data_jobs['company'].nunique()
print(f'{unique_company_count} companies combined for all 10 cities posted data related jobs on the job portal in last 120 days.')

3124 companies combined for all 10 cities posted data related jobs on the job portal in last 120 days.


### Finding Total number of jobs posted by each of the 10 cities in last 120 days.

In [47]:
# creating dataframe to hold city-wise count of data jobs posted in last 120 days in sorted order (ascending)

citywise_job_count = pd.DataFrame(data_jobs.groupby(['updated_city'])['company'].count()) \
                        .sort_values('company', ascending=False) \
                        .reset_index() \
                        .rename(columns = {'company': 'job_count'}).iloc[:10]
                       
citywise_job_count

,updated_city,job_count
0,Washington D.C.,2400
1,New York,1800
2,San Francisco,1374
3,Boston,1250
4,Seattle,1075
5,Atlanta,641
6,Denver,563
7,Austin,457
8,Raleigh,424
9,Columbus,299


In [48]:
# displaying city-wise count of data jobs posted in last 120 days in sorted order (ascending)

citywise_job_count.iplot(kind='bar', x='updated_city', y='job_count', 
                         yTitle='Number of data jobs posted', 
                         title='Top 10 US Tech Cities vs Number of data jobs posted in last 120 days',
                         theme='solar',
                         colors='yellow')

<hr>
<h4 align="center" style="color:black;"> Analysis: Top 10 US Tech Cities vs Number of data jobs posted in last 120 days </h4>
<h4 style="color:#666666;">Source: Indeed API </h4>
Based off of daily samples for 120 days (based of pubdates), we were able to find the total number of jobs posted by each one of the 10 "tech hubs" in the US for all data-related jobs (e.g. data scientist, BI analyst, data engineer, etc.). This was simply extracted by using Indeed's API and getting the value_counts() of these jobs (q or queries) in these cities (l or locations).
It is pretty clear to see that most of the growth in the last 4 months is happening in cities like DC (metro area), NYC and San Francisco. 
<hr>

### Finding the Companies that posted maximum data related jobs in each city.

In [49]:
# creating a dataframe to store number of jobs posted by companies grouped by city 

city_jobs = pd.DataFrame(data_jobs.groupby(['updated_city', 'company'])['company'].count()) \
                        .rename(columns = {'company': 'job_count'}) \
                        .reset_index() \
                        .sort_values(['updated_city','job_count'], ascending=False) \
                        .reset_index() \
                        .drop('index', axis=1)                        
city_jobs.head(3)

,updated_city,company,job_count
0,Washington D.C.,Ho-Chunk,169
1,Washington D.C.,"Compass, Inc.",115
2,Washington D.C.,Booz Allen Hamilton,70


In [50]:
# displaying top 50 companies for each city that posted maximum number of data jobs 

x1 = city_jobs[city_jobs['updated_city']=='Austin']['company'][:50].values
y1 = city_jobs[city_jobs['updated_city']=='Austin']['job_count'].values
trace1 = go.Bar(x=x1, y=y1, marker=dict(color='rgb(106,90,205)'), visible=True)

x2 = city_jobs[city_jobs['updated_city']=='Atlanta']['company'][:50].values
y2 = city_jobs[city_jobs['updated_city']=='Atlanta']['job_count'].values
trace2 = go.Bar(x=x2, y=y2, marker=dict(color='rgb(106,90,205)'), visible=False)

x3 = city_jobs[city_jobs['updated_city']=='Boston']['company'][:50].values
y3 = city_jobs[city_jobs['updated_city']=='Boston']['job_count'].values
trace3 = go.Bar(x=x3, y=y3, marker=dict(color='rgb(106,90,205)'), visible=False)

x4 = city_jobs[city_jobs['updated_city']=='Columbus']['company'][:50].values
y4 = city_jobs[city_jobs['updated_city']=='Columbus']['job_count'].values
trace4 = go.Bar(x=x4, y=y4, marker=dict(color='rgb(106,90,205)'), visible=False)

x5 = city_jobs[city_jobs['updated_city']=='Denver']['company'][:50].values
y5 = city_jobs[city_jobs['updated_city']=='Denver']['job_count'].values
trace5 = go.Bar(x=x5, y=y5, marker=dict(color='rgb(106,90,205)'), visible=False)

x6 = city_jobs[city_jobs['updated_city']=='New York']['company'][:50].values
y6 = city_jobs[city_jobs['updated_city']=='New York']['job_count'].values
trace6 = go.Bar(x=x6, y=y6, marker=dict(color='rgb(106,90,205)'), visible=False)

x7 = city_jobs[city_jobs['updated_city']=='Raleigh']['company'][:50].values
y7 = city_jobs[city_jobs['updated_city']=='Raleigh']['job_count'].values
trace7 = go.Bar(x=x7, y=y7, marker=dict(color='rgb(106,90,205)'), visible=False)

x8 = city_jobs[city_jobs['updated_city']=='San Francisco']['company'][:50].values
y8 = city_jobs[city_jobs['updated_city']=='San Francisco']['job_count'].values
trace8 = go.Bar(x=x8, y=y8, marker=dict(color='rgb(106,90,205)'), visible=False)

x9 = city_jobs[city_jobs['updated_city']=='Seattle']['company'][:50].values
y9 = city_jobs[city_jobs['updated_city']=='Seattle']['job_count'].values
trace9 = go.Bar(x=x9, y=y9, marker=dict(color='rgb(106,90,205)'), visible=False)

x10 = city_jobs[city_jobs['updated_city']=='Washington D.C.']['company'][:50].values
y10 = city_jobs[city_jobs['updated_city']=='Washington D.C.']['job_count'].values
trace10 = go.Bar(x=x10, y=y10, marker=dict(color='rgb(106,90,205)'), visible=False)


data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10]

updatemenus = list([
    dict(x = 1.00,
         y = 1.10,
         yanchor = 'top',
         showactive=True,
         active = 0,      
         buttons=list([   
            dict(label = "Austin",
                 method = "restyle",
                 args = [{"visible": [True, False, False, False, False, False, False, False, False, False]}]),
            dict(label = "Atlanta",
                 method = "update",
                 args = [{"visible": [False, True, False, False, False, False, False, False, False, False]}]),
            dict(label = "Boston",
                 method = "update",
                 args = [{"visible": [False, False, True, False, False, False, False, False, False, False]}]), 
            dict(label = "Columbus",
                 method = "update",
                 args = [{"visible": [False, False, False, True, False, False, False, False, False, False]}]),
            dict(label = "Denver",
                 method = "update",
                 args = [{"visible": [False, False, False, False, True, False, False, False, False, False]}]),
            dict(label = "New York",
                 method = "update",
                 args = [{"visible": [False, False, False, False, False, True, False, False, False, False]}]),
            dict(label = "Raleigh",
                 method = "update",
                 args = [{"visible": [False, False, False, False, False, False, True, False, False, False]}]),
            dict(label = "San Francisco",
                 method = "update",
                 args = [{"visible": [False, False, False, False, False, False, False, True, False, False]}]),
            dict(label = "Seattle",
                 method = "update",
                 args = [{"visible": [False, False, False, False, False, False, False, False, True, False]}]),
            dict(label = "Washington D.C.",
                 method = "update",
                 args = [{"visible": [False, False, False, False, False, False, False, False, False, True]}])
     ]))])


layout = dict(title="<b>City-Based: Top 50 Companies vs Number of Data Jobs Posted in last 120 Days</b>",
              showlegend=False,
              autosize=False,
              width=1000,
              height=800,
              yaxis=dict(title="Number of Data Jobs Posted", showgrid=True),
              updatemenus=updatemenus,
              plot_bgcolor='rgba(0,0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)

  

<hr>
<h4 align="center" style="color:black;"> Analysis: Citywise Top 50 Companies vs Number of Data Jobs Posted</h4>
<h4 style="color:#666666;">Source: Indeed API </h4>
By continuing the analysis of our data jobs dataframe, we're now looking at the aggregate of total number of data jobs for the top 50 companies, grouped by cities to determine which companies are hiring like mad in the past 4 months across all these tech hubs.  Unsurprisingly, names such as Deloitte, Accenture, Dell appear in most. Amazon has the lead in Seattle, Dell in Austin, and so on.
<hr>

### Finding top 20 Companies that posted highest number of data related jobs combined for all 10 cities.

In [51]:
# creating a dataframe that stores top 20 companies (combined for all cities) based on highest number of data job posted

sorted_city_jobs = city_jobs.sort_values('job_count', ascending=False)[:20] \
                            .reset_index() \
                            .drop('index', axis=1)

In [52]:
sorted_city_jobs 

,updated_city,company,job_count
0,Washington D.C.,Ho-Chunk,169
1,Seattle,"Amazon.com Services, Inc.",150
2,Washington D.C.,"Compass, Inc.",115
3,Seattle,Amazon.com Services LLC,104
4,New York,Oscar Health,73
5,New York,Disney Streaming Services,72
6,Seattle,Microsoft,71
7,Washington D.C.,Capital One - US,70
8,Washington D.C.,Booz Allen Hamilton,70
9,Washington D.C.,Leidos,68


In [53]:
# displaying top 20 companies (combined for all cities) based on highest number of data job posted

sorted_city_jobs.iplot(kind='scatter', x='company', y='job_count', 
                       yTitle='Number of data jobs posted', 
                       title='Top 20 Companies that posted highest number of data related jobs in last 120 days',
                       text='updated_city',
                       theme='solar',
                       colors='yellow')

<hr>
<h4 align="center" style="color:black;"> Analysis: Top 20 Companies that posted highest number of data-related jobs in last 120 days</h4>
<h4 style="color:#666666;">Source: Indeed API </h4>
This graph is rather a continuation of the previous one, but instead of looking at the top 50 companies by number of jobs and grouping by city, we're now looking at the top 20 with the highest number of data jobs to determine—across all tech hubs—which are the ones that have the most job openings in the past 120 days.  We were shocked to discover the Ho-Chunk is the company that has posted the most (based on DC), followed by Amazon (Seattle) and Compass (NYC).
<hr>

### Finding & Comparing the Number of jobs posted based on job title  

In [54]:
# creating the dataframe to store Data Engineer jobs

data_engineer_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains("engineer", case=False)].reset_index() 
data_engineer_jobs = pd.DataFrame(data_engineer_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'data_engineer_count'}) \
                                            .reset_index() \
                                            .sort_values('data_engineer_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

data_engineer_jobs

,updated_city,data_engineer_count
0,New York,481
1,San Francisco,424
2,Seattle,384
3,Washington D.C.,315
4,Boston,275
5,Austin,128
6,Denver,126
7,Atlanta,94
8,Raleigh,93
9,Columbus,40


In [55]:
# creating the dataframe to store Data Analyst/Analysis jobs

data_analyst_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains("analysis | analyst", case=False)].reset_index() 
data_analyst_jobs = pd.DataFrame(data_analyst_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'data_analyst_count'}) \
                                            .reset_index() \
                                            .sort_values('data_analyst_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

data_analyst_jobs

,updated_city,data_analyst_count
0,Washington D.C.,619
1,New York,267
2,Boston,255
3,San Francisco,221
4,Atlanta,154
5,Denver,127
6,Seattle,123
7,Austin,76
8,Raleigh,55
9,Columbus,43


In [56]:
# creating the dataframe to store Data Scientist/Science jobs

data_scientist_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains("scientist | science", case=False)].reset_index() 
data_scientist_jobs = pd.DataFrame(data_scientist_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'data_scientist_count'}) \
                                            .reset_index() \
                                            .sort_values('data_scientist_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

data_scientist_jobs

,updated_city,data_scientist_count
0,Washington D.C.,210
1,New York,192
2,San Francisco,143
3,Boston,108
4,Seattle,97
5,Atlanta,44
6,Austin,32
7,Denver,28
8,Raleigh,27
9,Columbus,21


In [57]:
# creating the dataframe to store BI jobs

BI_title_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains("business intelligence | BI", case=False)].reset_index()
BI_title_jobs = pd.DataFrame(BI_title_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'BI_title_count'}) \
                                            .reset_index() \
                                            .sort_values('BI_title_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

BI_title_jobs 

,updated_city,BI_title_count
0,Seattle,148
1,Boston,86
2,San Francisco,67
3,New York,56
4,Atlanta,55
5,Denver,52
6,Washington D.C.,52
7,Raleigh,32
8,Austin,28
9,Columbus,23


In [58]:
# creating the dataframe to store AI jobs

AI_title_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains("artificial intelligence | AI", case=False)].reset_index()
AI_title_jobs = pd.DataFrame(AI_title_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'AI_title_count'}) \
                                            .reset_index() \
                                            .sort_values('AI_title_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  
AI_title_jobs 

,updated_city,AI_title_count
0,Seattle,96
1,Boston,54
2,San Francisco,25
3,New York,21
4,Washington D.C.,18
5,Austin,14
6,Atlanta,9
7,Raleigh,9
8,Denver,7
9,Columbus,4


In [59]:
# creating the dataframe to store Machine Learning jobs

ML_title_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains('machine learning', case=False)].reset_index()
ML_title_jobs = pd.DataFrame(ML_title_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'ML_title_count'}) \
                                            .reset_index() \
                                            .sort_values('ML_title_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

ML_title_jobs 

,updated_city,ML_title_count
0,New York,170
1,Seattle,109
2,San Francisco,86
3,Boston,70
4,Washington D.C.,37
5,Austin,25
6,Denver,21
7,Atlanta,20
8,Raleigh,17
9,Columbus,9


In [60]:
# creating the dataframe to store Statistical/ Statistician jobs

stats_title_jobs = data_jobs.loc[data_jobs["jobtitle"].str.contains('statistical | statistician', case=False)].reset_index()
stats_title_jobs = pd.DataFrame(stats_title_jobs.groupby('updated_city')['jobtitle'].count()) \
                                            .rename(columns = {'jobtitle': 'stats_title_count'}) \
                                            .reset_index() \
                                            .sort_values('stats_title_count', ascending=False) \
                                            .reset_index() \
                                            .drop('index', axis=1)  

stats_title_jobs

,updated_city,stats_title_count
0,Boston,40
1,San Francisco,25
2,Raleigh,20
3,Washington D.C.,20
4,Atlanta,18
5,Seattle,7
6,Denver,6
7,New York,5
8,Austin,2
9,Columbus,1


In [61]:
# Visualizing the job counts based on job title

trace1 = go.Bar(x=data_analyst_jobs['updated_city'],
                    y=data_analyst_jobs['data_analyst_count'],
                    marker=dict(color='rgb(0,0,255)', line_color='rgb(0,0,255)'), 
                    name='Data Analyst Jobs')

trace2 = go.Bar(x=data_scientist_jobs['updated_city'],
                    y=data_scientist_jobs['data_scientist_count'],
                    marker=dict(color='rgb(95,158,160)', line_color='rgb(95,158,160)'), 
                    name='Data Scientist Jobs')

trace3 = go.Bar(x=data_engineer_jobs['updated_city'],
                    y=data_engineer_jobs['data_engineer_count'],
                    marker=dict(color='rgb(123,104,238)', line_color='rgb(123,104,238)'), 
                    name='Data Engineer Jobs')

trace4 = go.Bar(x=BI_title_jobs['updated_city'], 
                    y=BI_title_jobs['BI_title_count'], 
                    marker=dict(color='rgb(230,230,250)', line_color='rgb(230,230,250)'),
                    name='Business Intelligence Jobs')

trace5 = go.Bar(x=AI_title_jobs['updated_city'], 
                    y=AI_title_jobs['AI_title_count'], 
                    marker=dict(color='rgb(0,191,255)', line_color='rgb(0,191,255)'),
                    name='Artificial Intelligence Jobs')

trace6 = go.Bar(x=ML_title_jobs['updated_city'], 
                    y=ML_title_jobs['ML_title_count'], 
                    marker=dict(color='rgb(176,196,222)', line_color='rgb(176,196,222)'),
                    name='Machine Learning Jobs')

trace7 = go.Bar(x=stats_title_jobs['updated_city'],
                    y=stats_title_jobs['stats_title_count'], 
                    marker=dict(color='rgb(199,21,133)', line_color='rgb(199,21,133)'),
                    name='Statistician Jobs')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title="<b>City-Based: Job Title vs Number of Jobs Posted in Last 120 days</b>",
              showlegend=True,
              legend_orientation="h",
              autosize=False,
              width=1000,
              height=600,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Number of Jobs Posted", 
                         showgrid=False, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)


<hr>
<h4 align="center" style="color:black;"> Analysis: City-Based: Job Title vs Number of Jobs Posted in Last 120</h4>
<h4 style="color:#666666;">Source: Indeed API </h4>
There are more openings for Data Analyst and Data Engineer positions as compared to other Data Job Titles. Data Science positions are next in row. AI and Statistic jobs are relatively less in number.
<hr>

# <center><font size=6, font color='blue'> The MONEY Game !!!! </font></center>
<center><font color='red', font size=4><b>As long as my boss pretends my salary is high, I'll pretend that I have much work to do!</font></center></b>

### Analyzing the median and quartile values for minimum and maximum Data Job salaries

In [62]:
# visualizing the median and quartile values for minimum and maximum Data Job salaries

# minimum
trace1 = go.Box(x=jobs_sal_data['updated_city'],
                  y=jobs_sal_data['annual_min_salary'], 
                  name='Annual Minimum Salary for Data Jobs')

data = [trace1]

layout = dict(title="<b>Annual Minimum Data Job Salaries</b>",
              showlegend=False,
              legend_orientation="v",
              autosize=True,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Annual Min Data Salary", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)
iplot(fig)


# maximum
trace1 = go.Box(x=jobs_sal_data['updated_city'],
                  y=jobs_sal_data['annual_max_salary'], 
                  name='Annual Maximum Salary for Data Jobs')

data = [trace1]

layout = dict(title="<b>Annual Maximum Data Job Salaries</b>",
              showlegend=False,
              legend_orientation="v",
              autosize=True,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Annual Max Data Salary", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)
iplot(fig)

<hr>
<h4 align="center" style="color:black;"> Analysis: Annual Data Job Salaries </h4>
<h4 style="color:#666666;">Source: Indeed API and Web Scraping </h4>
This visualization represents the salary range based on the city. For most cities among this data, the max salaries or outliers, are significantly further away from the median than the minimum salaries. A feature of both visualizations is the median salary of a data job in San Francisco is higher. However, this is probably because of the higher living costs in the city themselves.
</hr>

### Comparing data job salaries to city and state annual median salary

In [63]:
state_median_sal

,AREA,ST,CITY_NAME,OCC_TITLE,OCC_GROUP,TOT_EMP,A_MEDIAN
0,6,CA,San Francisco,All Occupations,total,"17,007,690","42,430"
1,8,CO,Denver,All Occupations,total,"2,620,640","42,310"
2,11,DC,Washington D.C.,All Occupations,total,"712,370","71,690"
3,13,GA,Atlanta,All Occupations,total,"4,394,740","35,950"
4,25,MA,Boston,All Occupations,total,"3,571,360","48,680"
5,36,NY,New York,All Occupations,total,"9,385,620","44,990"
6,37,NC,Raleigh,All Occupations,total,"4,383,210","35,750"
7,39,OH,Columbus,All Occupations,total,"5,416,810","37,360"
8,48,TX,Austin,All Occupations,total,"12,113,810","37,100"
9,53,WA,Seattle,All Occupations,total,"3,259,150","46,100"


In [64]:
city_median_sal

,PRIM_STATE,CITY_NAME,OCC_TITLE,OCC_GROUP,TOT_EMP,A_MEDIAN
0,CA,San Francisco,All Occupations,total,"2,419,090","54,090"
1,CO,Denver,All Occupations,total,"1,478,110","45,420"
2,DC,Washington D.C.,All Occupations,total,"3,135,160","54,680"
3,GA,Atlanta,All Occupations,total,"2,682,910","39,080"
4,MA,Boston,All Occupations,total,"2,758,780","51,510"
5,NC,Raleigh,All Occupations,total,"629,570","39,460"
6,NY,New York,All Occupations,total,"9,498,420","47,080"
7,OH,Columbus,All Occupations,total,"1,051,570","39,300"
8,TX,Austin,All Occupations,total,"1,028,500","40,070"
9,WA,Seattle,All Occupations,total,"1,984,320","50,860"


In [65]:
data_jobs_median_sal

,updated_city,annual_min_median,annual_max_median
0,Atlanta,80000.0,110000.0
1,Austin,59004.0,73200.0
2,Boston,75000.0,120000.0
3,Columbus,65780.0,77508.8
4,Denver,63858.0,72624.0
5,New York,62862.0,66388.0
6,Raleigh,49000.0,60320.0
7,San Francisco,100000.0,135100.0
8,Seattle,44400.0,50000.0
9,Washington D.C.,75516.0,114533.0


In [66]:
# visualizing Annual Median Wages to compare Data Job Wages vs City Wages vs State Wages 

trace1 = go.Bar(x=city_median_sal['CITY_NAME'], 
                    y=city_median_sal['A_MEDIAN'], 
                    marker=dict(color='rgb(211,211,211)', line_color='rgb(211,211,211)'),
                    name='City Median Salary')

trace2 = go.Bar(x=state_median_sal['CITY_NAME'],
                  y=state_median_sal['A_MEDIAN'], 
                  marker=dict(color='rgb(95,158,160)', line_color='rgb(95,158,160)'),
                  name='State Median Salary')

trace3 = go.Bar(x=data_jobs_median_sal['updated_city'],
                  y=data_jobs_median_sal['annual_min_median'], 
                  marker=dict(color='rgb(147,112,219)', line_color='rgb(75,0,130)'),
                  name='Data Jobs Minimum Median Salary')

trace4 = go.Bar(x=data_jobs_median_sal['updated_city'],
                  y=data_jobs_median_sal['annual_max_median'], 
                  marker=dict(color='rgb(75,0,130)', line_color='rgb(75,0,130)'),
                  name='Data Jobs Maximum Median Salary')

data = [trace1, trace2, trace3, trace4]

layout = dict(title="<b>Annual Median Wages Comparison: Data Job Wages vs City Wages vs State Wages</b>",
              showlegend=True,
              legend_orientation="h",
              autosize=True,
              width=1000,
              height=600,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Annual Median Wage", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)


<hr>
<h4 align="center" style="color:black;"> Analysis: Annual Median Wages Comparison - Data Job Wages vs City Wages vs State Wages</h4>
<h4 style="color:#666666;">Source: Indeed API, Web-Scraping using RegEx (Indeed Job Descriptions) and Bureau of Labor Statistics </h4>
In most cities, Data Jobs have higher median salaries than the respecitve city and state’s median salary for all job types.  The outlier in the dataset is Seattle, this may be explained by the limited availiblity of salary data publically.  
<hr>

### Comparing Company Salary to Company Ratings to see if a correlation exists

In [67]:
jobs_sal_data.head(2)

,url,jobtitle,company,annual_min_salary,annual_max_salary,updated_city,state
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,Operations Analyst - Symitar System and Episys...,Austin Telco Federal Credit Union,50000.0,60000.0,Austin,TX
1,http://www.indeed.com/viewjob?jk=f6402472d770b...,Data Base Administrator IV,DEPARTMENT OF INFORMATION RESOURCES,75000.0,90000.0,Austin,TX


In [68]:
jobs_rating_data.head(2)

,jobtitle,company,rating,updated_city,state
0,Data Center Security Engineer,"Amazon Dev Center U.S., Inc.",3.6,Austin,TX
1,Senior Engineer - Data Science,The Home Depot,3.8,Austin,TX


In [69]:
# merging rating and salary data for the company

rating_and_sal_data = pd.merge(jobs_sal_data, jobs_rating_data, how='inner', on='company')


rating_and_sal_data = rating_and_sal_data[['url', 'jobtitle_x', 'company', 'annual_min_salary', 
                                          'annual_max_salary', 'updated_city_x', 'state_x', 'rating']] \
                        .rename(columns={'jobtitle_x': 'jobtitle', 'updated_city_x': 'updated_city', 'state_x': 'state'})

In [70]:
rating_and_sal_data.head(2) 

,url,jobtitle,company,annual_min_salary,annual_max_salary,updated_city,state,rating
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,Operations Analyst - Symitar System and Episys...,Austin Telco Federal Credit Union,50000.0,60000.0,Austin,TX,3.7
1,http://www.indeed.com/viewjob?jk=d3f99f0101182...,CPA - Data Analysis and Transparency Intern,COMPTROLLER OF PUBLIC ACCOUNTS,29568.0,29568.0,Austin,TX,3.0


In [71]:
# Visualizing company maximum salary vs company rating correlation

rating_and_sal_data.iplot(kind='scatter', x='rating', y='annual_max_salary', 
                          yTitle='Annual Max Data Job Salary', 
                          xTitle='Company Rating',
                          title='Company Annual Max Data Job Salary vs Company Rating',
                          text='company',
                          theme='solar',
                          colors='purple',
                          mode='markers')

<hr>
<h4 align="center" style="color:black;"> Analysis: Company Annual Max Data Job Salary vs Company Rating</h4>
<h4 style="color:#666666;">Source: Indeed API, Web-Scraping using RegEx (Indeed Job Header) </h4>
There does not appear to be a strong linear correlation between company’s rating and the salaries their employees are paid.  Most companies fall within a rating range of 3.5-4.5/5. Consistent salary data was difficult to obtain via scraping the Indeed job posts, the resulting data required additional cleaning which may skew the results. Some information that was not gathered for this exercise but may help to explain these ratings would be total compensation (insurance, stock options, etc.). 
<hr>

## Comparing Job Salary to Job's Minimum Experience Requirement to see if a correlation exists

In [72]:
jobs_sal_data.head(2)

,url,jobtitle,company,annual_min_salary,annual_max_salary,updated_city,state
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,Operations Analyst - Symitar System and Episys...,Austin Telco Federal Credit Union,50000.0,60000.0,Austin,TX
1,http://www.indeed.com/viewjob?jk=f6402472d770b...,Data Base Administrator IV,DEPARTMENT OF INFORMATION RESOURCES,75000.0,90000.0,Austin,TX


In [73]:
jobs_experience_data.head(2)

,jobtitle,company,exp,updated_city,state
0,Data Center Security Engineer,"Amazon Dev Center U.S., Inc.",3,Austin,TX
1,Senior Engineer - Data Science,The Home Depot,3,Austin,TX


In [74]:
# merging experience and salary data for the company

exp_and_sal_data = pd.merge(jobs_sal_data, jobs_experience_data, how='inner', on='company')

exp_and_sal_data = exp_and_sal_data[['url', 'jobtitle_x', 'company', 'annual_min_salary', 
                                     'annual_max_salary', 'updated_city_x', 'state_x', 'exp']] \
                        .rename(columns={'jobtitle_x': 'jobtitle', 'updated_city_x': 'updated_city', 'state_x': 'state'})                    

In [75]:
# exp_and_sal_data = exp_and_sal_data[exp_and_sal_data['exp'] <= 15]

In [76]:
exp_and_sal_data

,url,jobtitle,company,annual_min_salary,annual_max_salary,updated_city,state,exp
0,http://www.indeed.com/viewjob?jk=655ff099cd045...,Operations Analyst - Symitar System and Episys...,Austin Telco Federal Credit Union,50000.0,60000.0,Austin,TX,2
1,http://www.indeed.com/viewjob?jk=f6402472d770b...,Data Base Administrator IV,DEPARTMENT OF INFORMATION RESOURCES,75000.0,90000.0,Austin,TX,7
2,http://www.indeed.com/viewjob?jk=d3f99f0101182...,CPA - Data Analysis and Transparency Intern,COMPTROLLER OF PUBLIC ACCOUNTS,29568.0,29568.0,Austin,TX,8
3,http://www.indeed.com/viewjob?jk=f8eecb44bc4c5...,CPA - Senior Data Administrator (reopened),COMPTROLLER OF PUBLIC ACCOUNTS,90000.0,108000.0,Austin,TX,8
4,http://www.indeed.com/viewjob?jk=ce9e51e940d73...,CPA - Expenditure Audit Data Analyst,COMPTROLLER OF PUBLIC ACCOUNTS,60000.0,73200.0,Austin,TX,8
...,...,...,...,...,...,...,...,...
323,http://www.indeed.com/viewjob?jk=37894ee912a06...,Data Analytics Specialist,ALDI,62000.0,62000.0,Columbus,OH,3
324,http://www.indeed.com/viewjob?jk=3d1f148871f09...,Master Data Specialist,Greif,40000.0,55000.0,Columbus,OH,2
325,http://www.indeed.com/viewjob?jk=e9a663e9b3440...,Big data lead /sr .developer,Ace-stack LLC,156000.0,156000.0,Columbus,OH,2
326,http://www.indeed.com/viewjob?jk=62959dbf4adfe...,Big Data (Lead Developer),Ace-stack LLC,166400.0,166400.0,Columbus,OH,2


In [77]:
# Visualizing data job maximum salary vs job minimum experience requirement to find if a correlation exists

exp_and_sal_data.iplot(kind='scatter', x='exp', y='annual_max_salary', 
                          yTitle='Annual Max. Data Job Salary', 
                          xTitle='Min. Experience Required',
                          title='Max. Data Job Salary vs Minimum Experience Required for the Job',
                          text='jobtitle', 
                          theme='solar',
                          colors='green',
                          mode='markers')

<hr>
<h4 align="center" style="color:black;"> Analysis: Maximum Data Job Salary vs Minimum Experience Required for the Job</h4>
<h4 style="color:#666666;">Source: Indeed API, Web-Scraping using RegEx (Indeed Job Description) </h4>
There does not appear to be a strong linear correlation between maximum salary and minimum experience requirement for the job.  Most of the max salaries fall within an experience requirement of 1 to 5 years. Consistent experience data was difficult to obtain via scraping the Indeed job posts, the resulting data required additional cleaning which may skew the results.  
<hr>

## Comparing Data Job Salary to Average Apartment Rent

In [78]:
# merging median salary and rent dataframes

monthly_sal_and_rent = pd.merge(apt_rent_data, data_jobs_median_sal, how='inner', left_on='City', right_on='updated_city')

In [79]:
# calculating monthly median salaries from annual median salaries and assiging it to new columns

monthly_sal_and_rent =  monthly_sal_and_rent.assign(monthly_min_median_sal = monthly_sal_and_rent['annual_min_median']/12) \
                                            .assign(monthly_max_median_sal = monthly_sal_and_rent['annual_max_median']/12) \
                                            [['updated_city', 'monthly_min_median_sal', 'monthly_max_median_sal', '1br', '2br', '3br', '4br', 'Studio']] \
                                            .sort_values('monthly_min_median_sal', ascending=False)

In [80]:
monthly_sal_and_rent

,updated_city,monthly_min_median_sal,monthly_max_median_sal,1br,2br,3br,4br,Studio
7,San Francisco,8333.333333,11258.333333,2458,3088,4155,4941,2000
0,Atlanta,6666.666667,9166.666667,1039,1200,1575,1936,991
9,Washington D.C.,6293.000000,9544.416667,1361,1572,2073,2571,1295
2,Boston,6250.000000,10000.000000,1714,2125,2670,2943,1502
3,Columbus,5481.666667,6459.066667,752,970,1248,1452,636
4,Denver,5321.500000,6052.000000,1069,1353,1966,2277,875
5,New York,5238.500000,5532.333333,2158,2570,3314,3691,1943
1,Austin,4917.000000,6100.000000,1192,1471,1993,2399,983
6,Raleigh,4083.333333,5026.666667,1006,1162,1519,1857,816
8,Seattle,3700.000000,4166.666667,1345,1675,2431,2892,1175


In [81]:
# visualizing Monthly Minimum Median Wages to Average Rents for different apartment sizes

trace1 = go.Bar(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['monthly_min_median_sal'], 
                  width=0.7,
                  marker=dict(color='rgb(95,158,160)', line_color='rgb(65,105,225)'),
                  name='Data Jobs Minimum Median Salary')

trace2 = go.Scatter(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['4br'], 
                  marker=dict(color='rgb(255,0,255)', line_color='rgb(255,0,255)', size=10),
                  name='Avg Rent for 4 Br Apt')

trace3 = go.Scatter(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['3br'], 
                  marker=dict(color='rgb(230,230,250)', line_color='rgb(230,230,250)', size=10),
                  name='Avg Rent for 3 Br Apt')

trace4 = go.Scatter(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['2br'], 
                  marker=dict(color='rgb(124,252,0)', line_color='rgb(124,252,0)', size=10),
                  name='Avg Rent for 2 Br Apt')

trace5 = go.Scatter(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['1br'], 
                  marker=dict(color='rgb(0,255,255)', line_color='rgb(0,255,255)', size=10),
                  name='Avg Rent for 1 Br Apt')

trace6 = go.Scatter(x=monthly_sal_and_rent['updated_city'],
                  y=monthly_sal_and_rent['Studio'], 
                  marker=dict(color='rgb(255,215,0)', line_color='rgb(255,215,0)'),
                  name='Avg Rent for Studio Apt')

data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = dict(title="<b>Monthly Minimum Median Data Job Wages vs Apartment Average Rent</b>",
              showlegend=True,
              legend_orientation="v",
              autosize=True,
              width=1010,
              height=600,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Monthly Median Wage", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)


<hr>
<h4 align="center" style="color:black;"> Analysis: Monthly Minimum Median Data Job Wages vs Apartment Average Rent</h4>
<h4 style="color:#666666;">Source: Indeed API, Web-Scraping using RegEx (Indeed Job Descriptions) and Apartment List Rentonomics </h4>
The city with the highest median salaries for data science jobs, San Francisco, also has the most expensive apartment costs of all the cities in the dataset.  The relationship of high salary/high rent is not present for other large cities in the dataset.  For example, Atlanta has a large metropolitan population but has low rental cost relative to median salary. 
<hr>

## Comparing Data Job Salaries to Average House Price

In [82]:
# merging median salary and house price dataframes and selecting required columns.

sal_and_house_price = pd.merge(median_home_price_data, data_jobs_median_sal, how='inner', left_on='CITY', right_on='updated_city')
sal_and_house_price = sal_and_house_price[['updated_city', 'annual_min_median', 'annual_max_median', 1, 2, 3, 4, 5]] \
                            .sort_values('annual_min_median', ascending=False) \
                            .set_index('updated_city') 

In [83]:
sal_and_house_price

,annual_min_median,annual_max_median,1,2,3,4,5
updated_city,,,,,,,
San Francisco,100000.0,135100.0,NaN,NaN,1.037435e+06,1.283832e+06,1.637889e+06
Atlanta,80000.0,110000.0,201050.3333,178565.0000,1.979517e+05,2.892450e+05,4.183047e+05
Boston,75000.0,120000.0,350358.0000,382498.0000,4.577677e+05,6.168550e+05,8.774370e+05
Columbus,65780.0,77508.8,107595.3333,137180.0000,1.937777e+05,2.991533e+05,4.238153e+05
Denver,63858.0,72624.0,250056.6667,342350.0000,4.175047e+05,5.081567e+05,6.139150e+05
New York,62862.0,66388.0,391567.6667,340460.0000,4.192160e+05,5.310483e+05,7.738423e+05
Austin,59004.0,73200.0,258838.3333,310509.6667,2.977300e+05,3.856777e+05,5.653223e+05
Raleigh,49000.0,60320.0,173997.6667,173107.0000,2.331973e+05,3.603493e+05,4.889703e+05
Seattle,44400.0,50000.0,366461.3333,414984.0000,4.746977e+05,6.290253e+05,7.658623e+05


In [84]:
# using sklearn impute method to compute the missing values(NaN) for the above dataframe (for San Francisco)

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer,SimpleImputer 
from sklearn.linear_model import LinearRegression

In [85]:
# using sklearn impute method to compute the missing values(NaN) (for San Francisco)

lm = LinearRegression()
imp = IterativeImputer(estimator=lm)
imp_array = imp.fit_transform(sal_and_house_price)
imp_sal_and_house_price = pd.DataFrame(imp_array, index=sal_and_house_price.index, columns=sal_and_house_price.columns) 
imp_sal_and_house_price.reset_index(inplace=True) 

In [86]:
# no more NaN values in the dataframe

imp_sal_and_house_price

,updated_city,annual_min_median,annual_max_median,1,2,3,4,5
0,San Francisco,100000.0,135100.0,765678.797833,719293.794656,1.037435e+06,1.283832e+06,1.637889e+06
1,Atlanta,80000.0,110000.0,201050.333300,178565.000000,1.979517e+05,2.892450e+05,4.183047e+05
2,Boston,75000.0,120000.0,350358.000000,382498.000000,4.577677e+05,6.168550e+05,8.774370e+05
3,Columbus,65780.0,77508.8,107595.333300,137180.000000,1.937777e+05,2.991533e+05,4.238153e+05
4,Denver,63858.0,72624.0,250056.666700,342350.000000,4.175047e+05,5.081567e+05,6.139150e+05
5,New York,62862.0,66388.0,391567.666700,340460.000000,4.192160e+05,5.310483e+05,7.738423e+05
6,Austin,59004.0,73200.0,258838.333300,310509.666700,2.977300e+05,3.856777e+05,5.653223e+05
7,Raleigh,49000.0,60320.0,173997.666700,173107.000000,2.331973e+05,3.603493e+05,4.889703e+05
8,Seattle,44400.0,50000.0,366461.333300,414984.000000,4.746977e+05,6.290253e+05,7.658623e+05


In [87]:
# visualizing Annual Minimum Median Wages vs Median House Prices 

trace1 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price['annual_min_median'], 
                  marker=dict(color='rgb(255,69,0)', line_color='rgb(255,69,0)'),
                  name='Data Jobs Minimum Median Salary')

trace2 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price[5], 
                  marker=dict(color='rgb(25,25,112)', line_color='rgb(25,25,112)'),
                  name='Median Price for a 5 Bedroom House')

trace3 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price[4], 
                  marker=dict(color='rgb(0,0,255)', line_color='rgb(0,0,255)'),
                  name='Median Price for a 4 Bedroom House')

trace4 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price[3], 
                  marker=dict(color='rgb(0,191,255)', line_color='rgb(0,191,255)'),
                  name='Median Price for a 3 Bedroom House')

trace5 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price[2], 
                  marker=dict(color='rgb(135,206,235)', line_color='rgb(135,206,235)'),
                  name='Median Price for a 2 Bedroom House')

trace6 = go.Bar(x=imp_sal_and_house_price['updated_city'],
                  y=imp_sal_and_house_price[1], 
                  marker=dict(color='rgb(123,104,238)', line_color='rgb(123,104,238)'),
                  name='Median Price for a 1 Bedroom House')

data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = dict(title="<b>Annual Minimum Median Data Job Wages vs Median House Price</b>",
              showlegend=True,
              legend_orientation="h",
              autosize=True,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Median Wage and Median House Price", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)

<hr>
<h4 align="center" style="color:black;"> Analysis: Annual Minimum Median Data Job Wages vs Median House Price </h4>
<h4 style="color:#666666;">Source: Salary (Indeed API and web scraping)  & Zillow </h4>
This figure illustrates a similar trend that was seen in the median rent vs median salary information.  Cities that have higher median salaries have a more expensive real estate market.  San Francisco has the highest home prices as well as highest median salary of all the cities in the dataset.
</hr>

#  <br><center><font color='blue'> Ready to Relocate...Think Twice !!!! </font></center></br>

### Analyzing Number of Data Jobs vs City Crime Data for each of the 10 Cities

In [88]:
us_crime_data.head(2)

,city,total_pop,homs_sum,rape_sum,rob_sum,agg_ass_sum,violent_crime,violent_per_100k,homs_per_100k,rape_per_100k,rob_per_100k,agg_ass_per_100k,url
0,Atlanta,464710,94,170,1995,2944,5203,1119.622991,20.227669,36.581954,429.299993,633.513374,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
1,Austin,938728,23,487,929,2058,3497,372.525375,2.450124,51.878712,98.963704,219.232834,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...


In [89]:
citywise_job_count

,updated_city,job_count
0,Washington D.C.,2400
1,New York,1800
2,San Francisco,1374
3,Boston,1250
4,Seattle,1075
5,Atlanta,641
6,Denver,563
7,Austin,457
8,Raleigh,424
9,Columbus,299


In [90]:
# merging citywise job count data and US crime data. Selecting only required columns.

citywise_job_and_crime = pd.merge(citywise_job_count, us_crime_data, how='inner', left_on='updated_city', right_on='city')
citywise_job_and_crime = citywise_job_and_crime[['city', 'job_count', 'total_pop', 'violent_per_100k', 
                                                 'homs_per_100k', 'rape_per_100k', 'rob_per_100k', 'agg_ass_per_100k']]

In [91]:
citywise_job_and_crime

,city,job_count,total_pop,violent_per_100k,homs_per_100k,rape_per_100k,rob_per_100k,agg_ass_per_100k
0,Washington D.C.,2400,672228,1202.568176,24.098966,73.486972,506.375813,598.606425
1,New York,1800,8550861,585.765574,4.116545,26.242971,198.178873,357.227185
2,San Francisco,1374,863782,776.816373,6.135807,39.824863,417.929524,312.926178
3,Boston,1250,665258,706.793455,5.712070,36.076229,233.142630,431.862526
4,Seattle,1075,683700,598.654381,3.364049,21.061869,224.074887,350.153576
5,Atlanta,641,464710,1119.622991,20.227669,36.581954,429.299993,633.513374
6,Denver,563,682418,673.927124,7.766501,80.302688,180.241436,405.616499
7,Austin,457,938728,372.525375,2.450124,51.878712,98.963704,219.232834
8,Raleigh,424,438363,382.331538,3.878065,24.637116,154.666338,199.150020
9,Columbus,299,847745,546.272759,9.082920,95.075760,264.230400,177.883680


In [96]:
# visualizing City Crime Data vs Number of Data Jobs for the 10 cities 

trace1 = go.Bar(x=citywise_job_and_crime['city'],
                  y=citywise_job_and_crime['job_count'],
                  width=0.5,
                  marker=dict(color='rgb(95,158,160)', line_color='rgb(0,0,0)'),
                  name='Data Jobs Per City')

trace2 = go.Scatter(x=citywise_job_and_crime['city'], 
                    y=citywise_job_and_crime['homs_per_100k'], 
                    line=dict(color='rgb(255,0,255)', width=3),
                    name='homs_sum_per_capita')

trace3 = go.Scatter(x=citywise_job_and_crime['city'],
                  y=citywise_job_and_crime['rape_per_100k'], 
                  line=dict(color='rgb(0,255,255)', width=3),
                  name='rape_sum_per_capita')

trace4 = go.Scatter(x=citywise_job_and_crime['city'],
                  y=citywise_job_and_crime['rob_per_100k'],
                  line=dict(color='rgb(124,252,0)', width=3),
                  name='rob_sum_per_capita')

trace5 = go.Scatter(x=citywise_job_and_crime['city'],
                  y=citywise_job_and_crime['agg_ass_per_100k'],
                  line=dict(color='rgb(230,230,250)', width=3),
                  name='agg_ass_sum_per_capita')

trace6 = go.Scatter(x=citywise_job_and_crime['city'],
                  y=citywise_job_and_crime['violent_per_100k'],
                  line=dict(color='rgb(0,191,255)', width=3),
                  name='violent_crime_per_capita')

data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = dict(title="<b>Number of Data Jobs vs Per Capita City Crime Rate</b>",
              showlegend=True,
              legend_orientation="v",
              autosize=True,
              width=950,
              height=600,
              xaxis=dict(showgrid=False),
              yaxis=dict(title="Number of Data Jobs and Per Capita Crime", 
                         showgrid=True, 
                         autorange=True,
                         zeroline=False,
                         showline=False,
                         ticks='',
                         showticklabels=True),
              plot_bgcolor='rgba(0,0,0)',
              paper_bgcolor='rgba(0,0,0,0)',
          )

fig=dict(data=data, layout=layout)

iplot(fig)


<hr>
<h4 align="center" style="color:black;"> Analysis: Number of Data Jobs vs Per Capita City Crime Rate </h4>
<h4 style="color:#666666;">Source: Indeed API and Data.world(FBI Data)</h4>
One of the more interesting observations from the graph is the amount of violent crime compared to data jobs. There is almost twice as much crime as there is data jobs listed in Atlanta. Conversely, the city with the lowest crime rate in proportion to the highest job count is New York City. The two most prevalent crimes other than violent crimes within the cities is aggravated assault and robbery.
</hr>

## Uploading output dataframes to SQL database on cloud server and reading the SQL tables back into the dataframes

In [95]:
# uploading the dataframes to sql database (on cloud using proxy server)

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

USER = "root"
PASSWORD = "Attrye@123" #your instance password
HOST = "127.0.0.1"  
PORT = "3306"  

DATABASE = "jobs_db"  

TABLENAME1 = "data_jobs"  
TABLENAME2 = "city_jobs"
TABLENAME3 = "citywise_job_count"  
TABLENAME4 = "data_jobs_median_sal"  
TABLENAME5 = "monthly_sal_and_rent"
TABLENAME6 = "sal_and_house_price"
TABLENAME7 = "citywise_job_and_crime"


engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}")
try:
    engine.execute(f"CREATE DATABASE {DATABASE}")
except ProgrammingError:
    pass

engine.execute(f"USE {DATABASE}")

engine.execute(f"DROP TABLE IF EXISTS {TABLENAME1}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME2}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME3}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME4}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME5}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME6}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME7}")

data_jobs.to_sql(name=TABLENAME1, con=engine, index=False)
city_jobs.to_sql(name=TABLENAME2, con=engine, index=False)
citywise_job_count.to_sql(name=TABLENAME3, con=engine, index=False)
data_jobs_median_sal.to_sql(name=TABLENAME4, con=engine, index=False)
monthly_sal_and_rent.to_sql(name=TABLENAME5, con=engine, index=False)
sal_and_house_price.to_sql(name=TABLENAME6, con=engine, index=False)
citywise_job_and_crime.to_sql(name=TABLENAME7, con=engine, index=False)



OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query')
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# reading the tables from sql database back into pandas datafarmes

data_jobs_sql_table = pd.read_sql("SELECT * FROM jobs_db.data_jobs", engine)
city_jobs_sql_table = pd.read_sql("SELECT * FROM jobs_db.city_jobs", engine)
citywise_job_count_sql_table = pd.read_sql("SELECT * FROM jobs_db.citywise_job_count", engine)
monthly_sal_and_rent_sql_table = pd.read_sql("SELECT * FROM jobs_db.monthly_sal_and_rent", engine)
data_jobs_median_sal_sql_table = pd.read_sql("SELECT * FROM jobs_db.data_jobs_median_sal", engine)
sal_and_house_price_sql_table = pd.read_sql("SELECT * FROM jobs_db.sal_and_house_price", engine)
citywise_job_and_crime_sql_table = pd.read_sql("SELECT * FROM jobs_db.citywise_job_and_crime", engine)


In [ ]:
data_jobs_sql_table.head(2)

In [ ]:
city_jobs_sql_table.head(3) 

In [ ]:
citywise_job_count_sql_table.head(3)

In [ ]:
monthly_sal_and_rent_sql_table.head(3)

In [ ]:
data_jobs_median_sal_sql_table.head(3)

In [ ]:
sal_and_house_price_sql_table.head(3)

In [ ]:
citywise_job_and_crime_sql_table.head(3)

# Conclusion

The current job market for Data Jobs is quite promising. Big as well as small - all sorts of companies have many data related jobs posted.
Salaries for Data Jobs are quite good as well as compared to the city and state median salaries for all occupations. 

As there is always a scope for improvement, we would want to explore historical data for data jobs market (if availabe) and use advanced RegEx for web scraping.